In [1]:
from antelope_foreground import ForegroundCatalog
from antelope import enum  # utility for listing iterable results for interactive use

In [2]:
def lca_init():
    cat = ForegroundCatalog('catalog', strict_clookup=False, quell_biogenic_co2=True)
    cat.lcia_engine.add_synonym('number of items', 'Items')
    cat.lcia_engine.add_synonym('number of items', 'Count')
    cat.lcia_engine.get_canonical('volume')['UnitConversion']['mL'] = 1e6
    return cat

cat = lca_init()
mass = cat.get_canonical('mass')
if 'coffee_demo' in cat.foregrounds:
    cat.delete_foreground('coffee_demo')  # cleanup for fresh start

merging Emissions into Emissions
merging Resources into Resources
Loading JSON data from /data/GitHub/Antelope/demo/catalog/reference-quantities.json:
local.qdb: /data/GitHub/Antelope/demo/catalog/reference-quantities.json
local.qdb: Setting NSUUID (False) 77833297-6780-49bf-a61a-0cb707dce700
local.qdb: /data/GitHub/lca-tools/lcatools/qdb/data/elcd_reference_quantities.json
26 new quantity entities added (26 total)
6 new flow entities added (6 total)


## origins and interfaces
A catalog maps queries to resources. The queries come in terms of an _origin_ (which is an "agreed-on data source") and a _request_.  Different classes of requests are handled by different _interfaces_ which include:

 * index : search, retrieve, metadata
 * exchange : quantitative information about related flows through processes
 * quantity : quantitative information about related quantities (measures) of flows
 * background : graph ordering, matrix construction + calculation
 * foreground : model building
 
`qdb` is the local quantity database.

In [3]:
cat.show_interfaces()

ecoinvent.2.2 [basic, exchange]
ecoinvent.3.7.1.cutoff [background, basic, exchange, index]
ecoinvent.3.8.cutoff [background, basic, exchange, index]
lcacommons.useeio.2.0.1 [background, basic, exchange, index]
lcacommons.uslci.fy20.q4 [basic, exchange]
lcacommons.uslci.fy21.q1 [background, basic, exchange, index, quantity]
local.lcia.traci.2.1 [basic, index, quantity]
local.qdb [index, quantity]
openlca.lcia.2.1.1 [basic, index, quantity]
openlca.refdata.2.1 [basic, index, quantity]
u-cuenca.ecoinvent.2.2 [background, basic, exchange, index]


## Foregrounds
Foregrounds are the basic container for organizing models.

In [4]:
fg = cat.create_foreground('coffee_demo')

QQQQQQQQQQQQQQQQQQ coffee_demo QQQQQQQQQQQQQQQQQQ
Found Antelope providers:
antelope_core.providers:IlcdArchive
antelope_core.providers:IlcdLcia
antelope_core.providers:EcospoldV2Archive
antelope_core.providers:EcospoldV1Archive
antelope_core.providers:EcoinventLcia
antelope_core.providers:OpenLcaJsonLdArchive
antelope_core.providers:Traci21Factors
antelope_background.providers:TarjanBackground
antelope_background.providers:Background
antelope_foreground.providers:AntelopeV1Client
antelope_foreground.providers:LcForeground
coffee_demo: /data/GitHub/Antelope/demo/catalog/coffee_demo
coffee_demo: Setting NSUUID (None) None
Loading /data/GitHub/Antelope/demo/catalog/coffee_demo
0 new fragment entities added (0 total)


## Helper class
This class performs / simplifies contextual tasks that casual users would perform graphically

In [5]:
tr = str.maketrans(' ', '_', ',[]()*&^%$#@')

def _flow_to_ref(name):
    n = name.translate(tr).lower()
    if n.startswith('flow_'):
        fl = n
    else:
        fl = 'flow_' + n
    return fl


class QuickModelBuilder(object):
    """
    Convenience class for building models.  Performs the following tasks:
    
     - maintain a list of references to background processes or 'terminations' in active use (or via query)
     - create or retrieve flows an
     - create fragments comprising those flows
     - terminate those fragments to background processes
    """
    
    def set_terms(self, terms):
        if terms:
            for k, v in terms.items():
                self._terms[k] = self.fg.catalog_ref(*v)
    
    def terms(self, term):
        return self._terms[term]

    def __init__(self, fg, terms=None):
        self._fg = fg
        self._terms = {}
        self.set_terms(terms)

    @property
    def fg(self):
        return self._fg

    def _new_reference_fragment(self, flow, direction, external_ref):
        frag = self.fg[external_ref]
        if frag is None:
            frag = self.fg.new_fragment(flow, direction, external_ref=external_ref)

        return frag

    def new_link(self, flow_name, ref_quantity, direction, amount=None, units=None, flow_ref=None, parent=None, name=None,
                 stage=None,
                 prefix='frag',
                 balance=None):
        """
        Create or retrieve a flow, and create a directed link [fragment] comprising that flow.  
        
        The link is either a reference fragment (no parent supplied) or a child fragment (yes parent supplied).
        
        Reference fragments must be named (auto-generated from flow name if none is given)
        
        examples:
        
        mass = cat.get_canonical('mass')
        
        k = self.new_link('X-2000 Widget', 'number of items', 'Output', amount=1.0, name='x2k-widget')
        k.flow.characterize(mass, 450.0)
        l = self.new_link('Widget goop', 'mass', 'Input', amount=434.0, units='kg', parent=k)
        m = self.new_link('Widget chunks', 'mass', 'Input', balance=True, parent=k)

        When this fragment is traversed, the inflow of widget chunks will be dynamically computed to be 16 kg.

        l.terminate(self.terms('widget-goop-production'))
        m.terminate(self.terms('widget-chunk-production'))
        
        This designates some other model as the source of information about the two respective inflows.

        :param flow_name:
        :param ref_quantity: of flow
        :param direction: of fragment with respect to parent (or self if no parent)
        :param amount:
        :param units:
        :param flow_ref:
        :param parent:
        :param name:
        :param stage:
        :param prefix: what to add to the auto-name in order to
        :param balance: direction='balance' should be equivalent;; direction is irrelevant under balance
        :return:
        """
        if flow_ref is None:
            flow_ref = _flow_to_ref(flow_name)

        flow = self.fg.add_or_retrieve(flow_ref, ref_quantity, flow_name)

        external_ref = name or None
        if parent is None:
            if flow_ref.startswith('flow_'):
                auto_name = flow_ref[5:]
            else:
                auto_name = '%s_%s' % (prefix, flow_ref)
            external_ref = external_ref or auto_name

            frag = self._new_reference_fragment(flow, direction, external_ref)
            self.fg.observe(frag, exchange_value=amount, units=units)
        else:
            if direction == 'balance':
                balance = True
            if balance:
                frag = self.fg.new_fragment(flow, direction, parent=parent, balance=True)
            else:
                frag = self.fg.new_fragment(flow, direction, value=1.0, parent=parent, external_ref=external_ref)
                self.fg.observe(frag, exchange_value=amount, units=units)

        if stage:
            frag['StageName'] = stage

        return frag


In [6]:
# "terminations" or "provider" activities, drawn from the ecoinvent database.
# these were obtained through manual [expert] review and data selection
TERMS = {
    'coffee_green': ['ecoinvent.3.8.cutoff', '40cd93ba-b97f-43e5-8bbd-f86e34cf1d4d'],
    'elec_wecc': ['ecoinvent.3.8.cutoff', '56c40e26-3740-4973-916e-3999d997ea60'],
    'ng_mj': ['ecoinvent.3.8.cutoff', '242162aa-17fb-4e5d-923a-899b5b7bb864'],
    'ng_mj_high': ['ecoinvent.3.8.cutoff', '44e62980-8489-4a80-8ecb-fca6a12e2d05'],
    'tissue_paper': ['ecoinvent.3.8.cutoff', '6f123135-58e0-4e4c-b556-19d5e71ea15e'],
    'sugar': ['ecoinvent.3.8.cutoff', 'e24d837f-eabb-41df-b282-c97af565b6ca'],
    'soap': ['ecoinvent.3.8.cutoff',  '21abc849-d8aa-4151-b063-b10df257abfc'],
    'tap_water': ['ecoinvent.3.8.cutoff', 'ac596c5b-7779-4a51-9c5f-9ec2e0025d6d'],
    'landfill': ['ecoinvent.3.8.cutoff', '37fecf1c-c830-4bff-a8fe-76aa79d7ff2d']
}


In [7]:
B = QuickModelBuilder(fg, terms=TERMS)

QQQQQQQQQQQQQQQQQQ ecoinvent.3.8.cutoff QQQQQQQQQQQQQQQQQQ
Loading JSON data from /data/LCI/aws-data/ecoinvent.3.8.cutoff/index/json/8aaaf7f46f8d75939cadb7c4e297ebd831eff443.json.gz:
ecoinvent.3.8.cutoff.index.20220509: /data/LCI/aws-data/ecoinvent.3.8.cutoff/index/json/8aaaf7f46f8d75939cadb7c4e297ebd831eff443.json.gz
ecoinvent.3.8.cutoff.index.20220509: Setting NSUUID (None) None
ecoinvent.3.8.cutoff.index.20220509: <source removed>
ecoinvent.3.8.cutoff: /data/LCI/aws-data/ecoinvent.3.8.cutoff/index/json/8aaaf7f46f8d75939cadb7c4e297ebd831eff443.json.gz
ecoinvent.3.8.cutoff: /data/LCI/aws-data/ecoinvent.3.8.cutoff/exchange/EcospoldV2Archive/ecoinvent 3.8_cutoff_ecoSpold02.zip
20 new quantity entities added (20 total)
5463 new flow entities added (5463 total)
17910 new process entities added (17910 total)
Applying stored configuration
Applying quantity hint EcoSpold Quantity m2 -> Area
Applying context hint ecoinvent.3.8.cutoff.index.20220509:water => to water
Applying context hint ecoi

## Model Construction
Here we actually specify the models by building connected links. We start with one reference fragment, and then add child fragments that use it as the parent. Each child fragment is terminated to a process named above in `TERMS`.

In [8]:
## coffee roasting
# https://link.springer.com/article/10.1007/s11367-019-01719-2/tables/2
roast = B.new_link('Coffee, roasted', 'mass', 'Output', 1000.0)  # default unit is kg
B.new_link('Coffee, green', 'mass', 'Input', 1152.0, parent=roast).terminate(B.terms('coffee_green'))
B.new_link('Electricity', 'kWh', 'Input', 275, units='MJ', parent=roast).terminate(B.terms('elec_wecc'))
B.new_link('Heat, natural gas', 'net calorific value', 'Input', 1702, units='MJ',
              parent=roast).terminate(B.terms('ng_mj_high'))
B.new_link('Water', 'mass', 'Input', 384, units='kg', parent=roast).terminate(B.terms('tap_water'))


Naming fragment 2a2707a5-2b90-48da-9665-61546a906a9b -> coffee_roasted


In [9]:
roast.show()

( ** ref) -<- 2a270 -<- -O    [   1 kg] Coffee, roasted {coffee_roasted}
LcFragment Entity (ref coffee_roasted)
origin: coffee_demo
reference: None
     Name: Coffee, roasted
  Comment: 
StageName: 
Exchange values: 
              Cached: 1
            Observed: 1000

Balance flow: False
Terminations: 
            Scenario  Termination
                None: -O   Foreground


## Observed flows
The software distinguishes between "observed" and "unobserved" exchanges.  Observation is essentially the act of measuring (or otherwise corroborating) the numerical amount of each exchange.  Unobserved exchanges express the structure of the graph (and may include data); observed exchanges express data.  

Alternate observations can be made and tagged to different scenarios, which are specified during traversal. 

The model is suitably visualized as an ASCII-art tree diagram.  The `B*` indicate background processes.

In [10]:
# unobserved 
roast.show_tree()

   -<--O   2a270 [       1 kg] Coffee, roasted
    [   1 unit] Coffee, roasted
       |        Stage: heat production, natural gas, at boiler condensing modulating >100kW
       | -<--B*  f5ffb [       1 MJ] Heat, natural gas
       |        Stage: market for coffee, green bean
       | -<--B*  718a8 [       1 kg] Coffee, green
       |        Stage: market for electricity, low voltage
       | -<--B*  e0179 [       1 kWh] Electricity
       |        Stage: market for tap water
       | -<--B*  c222f [       1 kg] Water
       x 


In [11]:
# observed
roast.show_tree(observed=True)

   -<--O   2a270 [   1e+03 kg] Coffee, roasted
    [   1 unit] Coffee, roasted
       |        Stage: heat production, natural gas, at boiler condensing modulating >100kW
       | -<--B*  f5ffb [ 1.7e+03 MJ] Heat, natural gas
       |        Stage: market for coffee, green bean
       | -<--B*  718a8 [ 1.15e+03 kg] Coffee, green
       |        Stage: market for electricity, low voltage
       | -<--B*  e0179 [    76.4 kWh] Electricity
       |        Stage: market for tap water
       | -<--B*  c222f [     384 kg] Water
       x 


## More model components
The `#` indicates a termination to another fragment (sub-model); in this case coffee roasting from above

In [12]:
# grinding coffee requires roast coffee and electricity
# note the balance flow
cg = B.new_link('Coffee, ground', 'mass', 'Output', 1.0)
B.new_link('Coffee, roasted', 'mass', 'Input', balance=True, parent=cg).terminate(roast)
B.new_link('Electricity, grinding', 'kWh', 'Input', 0.35, parent=cg).terminate(B.terms('elec_wecc'))


cg.show_tree(True)

Naming fragment 5633d414-e907-48f4-b68e-1485b42df9b4 -> coffee_ground
   -<--O   5633d [       1 kg] Coffee, ground
    [   1 unit] Coffee, ground
       | -<--#:: 2bd01 (=      0 kg) Coffee, roasted
       |        Stage: market for electricity, low voltage
       | -<--B*  6df5e [*   0.35 kWh] Electricity, grinding
       x 


In [13]:
# brewing requires ground coffee, a fraction of a paper filter, electricity, water, and disposal
brew = B.new_link('Coffee, 1 cup', 'volume', 'Output', 180, units='mL')
brew.flow.characterize(mass, 1000.0)

B.new_link('Coffee, ground', 'mass', 'Input', 0.010, parent=brew, stage='Ground Coffee').terminate(cg, descend=False)
filt = B.new_link('Coffee filter', 'Count', 'Input', 0.2, parent=brew)
filt.flow.characterize(mass, 0.0017)
B.new_link('New Coffee Filter', 'mass', 'Input', 0.0017, parent=filt, stage='Filter').terminate(B.terms('tissue_paper'))

B.new_link('Electricity, at user', 'kWh', 'Input', 0.175, parent=brew, stage='Electricity').terminate(B.terms('elec_wecc'))
water = B.new_link('Water, at user', 'volume', 'Input', 190, units='mL', parent=brew, stage='Tap water')

# We are going to calculate the weight of the waste flow using a mass balance... in order to do that,
# we need the water flows (tap water and coffee) to be characterized w/r/t mass
water.flow.characterize(mass, 1000.0)
water.terminate(B.terms('tap_water'))

B.new_link('Spent coffee grounds', 'mass', 'Output', balance=True, parent=brew, stage='Waste to landfill').terminate(B.terms('landfill'))  # OMG 'balance' is a direction




brew.show_tree(True)

Naming fragment 61e29fef-e562-4086-8685-de813fd4519a -> coffee_1_cup
   -<--O   61e29 [*0.00018 m3] Coffee, 1 cup
    [   1 unit] Coffee, 1 cup
       | -<--O   84d87 [*    0.2 Item(s)] Coffee filter
       |  [   1 unit] Coffee filter
       |     |        Stage: Filter
       |     | -<--B*  b40d8 [* 0.0017 kg] New Coffee Filter
       |     x 
       |        Stage: Electricity
       | -<--B*  0ec55 [*  0.175 kWh] Electricity, at user
       |        Stage: Ground Coffee
       | -<--#   b3b77 [*   0.01 kg] Coffee, ground
       |        Stage: Tap water
       | -<--B*  82f02 [*0.00019 m3] Water, at user
       |        Stage: Waste to landfill
       | =>=-B*  fc1c7 (=      0 kg) Spent coffee grounds
       x 


In [14]:
coffee = B.new_link('Morning coffee', 'Items', 'Output', 1.0)
B.new_link('Coffee', 'volume', 'Input', 330, units='mL', parent=coffee).terminate(brew)
B.new_link('Sugar', 'mass', 'Input', 0.015, parent=coffee, stage='Sugar').terminate(B.terms('sugar'))


Naming fragment 21dbd07e-c5dc-4ac9-9230-8c44d448b8ed -> morning_coffee


In [15]:
coffee.show_tree(observed=True)

   -<--O   21dbd [       1 Item(s)] Morning coffee
    [   1 unit] Morning coffee
       | -<--#:: 39ac2 [ 0.00033 m3] Coffee
       |        Stage: Sugar
       | -<--B*  8233b [   0.015 kg] Sugar
       x 


## Traversal
Once a model is constructed as a tree, the _activity levels_ of each node can be determined via traversal.  The magnitude of each node is equal to the magnitude of the parent node, times the exchange value of the link.

In [16]:
# unobserved traversal
_=enum(coffee.traverse())

 [00] 21dbd           1 [ Input] -O   morning_coffee
 [01] 39ac2           1 [ Input] -#:: coffee_1_cup
 [02] 61e29           1 [ Input] -O   coffee_1_cup
 [03] b3b77           1 [ Input] -#   coffee_ground
 [04] 84d87           1 [ Input] -O   Coffee filter
 [05] b40d8           1 [ Input] -B*  market for tissue paper [GLO]
 [06] 0ec55           1 [ Input] -B*  market for electricity, low voltage [US-WECC]
 [07] 82f02           1 [ Input] -B*  market for tap water [RoW]
 [08] fc1c7           1 [Output] -B*  treatment of municipal solid waste, sanitary landfill [RoW]
 [09] 8233b           1 [ Input] -B*  market for sugar, from sugarcane [GLO]


In [17]:
# observed traversal
_=enum(coffee.traverse(True))

 [00] 21dbd           1 [ Input] -O   morning_coffee
 [01] 39ac2     0.00033 [ Input] -#:: coffee_1_cup
 [02] 61e29     0.00033 [ Input] -O   coffee_1_cup
 [03] b3b77      0.0183 [ Input] -#   coffee_ground
 [04] 84d87       0.367 [ Input] -O   Coffee filter
 [05] b40d8    0.000623 [ Input] -B*  market for tissue paper [GLO]
 [06] 0ec55       0.321 [ Input] -B*  market for electricity, low voltage [US-WECC]
 [07] 82f02    0.000348 [ Input] -B*  market for tap water [RoW]
 [08] fc1c7      0.0373 [Output] -B*  treatment of municipal solid waste, sanitary landfill [RoW]
 [09] 8233b       0.015 [ Input] -B*  market for sugar, from sugarcane [GLO]


Note the magnitude of the balance flow "treatment of municipal solid waste, sanitary landfill" of 0.0373 kg:
 - our "coffee, 1 cup" equals 180 mL of coffee
 - our "morning coffee" functional unit is 330 mL => "coffee, 1 cup" __node weight = 330/180 = 1.833__
 - the filter weight is __1.833__ (cups/FU) * 0.2 (filters/cup) * 0.0017 (weight/filter) = 0.6233 g
 - the coffee weight is __1.833__ (cups/FU) * 10 (g grounds/cup) = 18.33 g
 - the water weight is __1.833__ (cups/FU) * 190 mL = 348.3 g
 - the coffee out is 330 g
 - the electricity has no mass

So the balance flow to landfill is (348.3 + 18.33 + 0.6233 - 330) = 37.25 g

## LCIA
LCIA information is stored in the quantity interface.  

In [18]:
# don't mind the XLSX warnings- the TRACI spreadsheet contains fancy visual features that are not supported
qs = enum(cat.query('local.lcia.traci.2.1').lcia_methods())

QQQQQQQQQQQQQQQQQQ local.lcia.traci.2.1 QQQQQQQQQQQQQQQQQQ
local.lcia.traci.2.1: /data/LCI/TRACI/traci_2_1_2014_dec_10_0.xlsx
local.lcia.traci.2.1: Setting NSUUID (True) 150e35c3-ac4a-485b-826b-a41807ddf43a
Loading workbook /data/LCI/TRACI/traci_2_1_2014_dec_10_0.xlsx


/data/virtualenvs/antelope/lib/python3.9/site-packages/openpyxl/reader/drawings.py:59: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)


Applying stored configuration
Applying context hint local.lcia.traci.2.1:water => to water
Applying context hint local.lcia.traci.2.1:air => to air
Applying configuration to Traci21Factors with 11 entities at /data/LCI/TRACI/traci_2_1_2014_dec_10_0.xlsx
 [00] [local.lcia.traci.2.1] Acidification Air [kg SO2 eq] [LCIA]
 [01] [local.lcia.traci.2.1] Ecotoxicity, freshwater [CTUeco] [LCIA]
 [02] [local.lcia.traci.2.1] Eutrophication Air [kg N eq] [LCIA]
 [03] [local.lcia.traci.2.1] Eutrophication Water [kg N eq] [LCIA]
 [04] [local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA]
 [05] [local.lcia.traci.2.1] Human Health Particulates Air [PM2.5 eq] [LCIA]
 [06] [local.lcia.traci.2.1] Human health toxicity, cancer [CTUcancer] [LCIA]
 [07] [local.lcia.traci.2.1] Human health toxicity, non-cancer [CTUnoncancer] [LCIA]
 [08] [local.lcia.traci.2.1] Ozone Depletion Air [kg CFC-11 eq] [LCIA]
 [09] [local.lcia.traci.2.1] Smog Air [kg O3 eq] [LCIA]


/data/virtualenvs/antelope/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [19]:
qs[4].show()

QuantityRef catalog reference (Global Warming Air)
origin: local.lcia.traci.2.1
   UUID: 9304410a-9dfc-3c85-8a30-c548893097c3
   Name: Global Warming Air
Comment: 
referenceUnit: kg CO2 eq
==Local Fields==
           Indicator: kg CO2 eq
          local_Name: Global Warming Air
       local_Comment: 
local_UnitConversion: {'kg CO2 eq': 1.0}
        local_Method: TRACI 2.1
      local_Category: Global Warming Air
     local_Indicator: kg CO2 eq


## LCIA Computation
Each node has a _unit score_ based on what the node is terminated to.  The inner product of node weights and unit scores equals the LCIA result.  Information is only retrieved when required, and computational (LCI) results and flow characterization factors are cached.

In [20]:
fg.get('morning_coffee').fragment_lcia(qs[4]).show_components()

QQQQQQQQQQQQQQQQQQ ecoinvent.3.8.cutoff QQQQQQQQQQQQQQQQQQ
ecoinvent.3.8.cutoff: /data/LCI/aws-data/ecoinvent.3.8.cutoff/background/TarjanBackground/8aaaf7f46f8d75939cadb7c4e297ebd831eff443.mat
ecoinvent.3.8.cutoff: Setting NSUUID (False) None
Loading JSON data from /data/LCI/aws-data/ecoinvent.3.8.cutoff/background/TarjanBackground/8aaaf7f46f8d75939cadb7c4e297ebd831eff443.mat.ordering.json.gz:
completed 63 iterations
Imported 91 factors for [local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA]
completed 67 iterations
completed 64 iterations
completed 51 iterations
completed 64 iterations
completed 60 iterations
completed 65 iterations
[local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA] kg CO2 eq
------------------------------------------------------------
     0.129 =      0.321 x      0.403 | 0ec55       0.321 [ Input] -B*  market for electricity, low voltage [US-WECC]
     0.121 =     0.0183 x       6.61 | b3b77      0.0183 [ Input] -#   coffee_ground
   0.00794 =   

## Aggregation
You can control the information revealed in traversal and fragment LCIA by specifying whether the traversal "descends" through each link.  The descend setting is indicated by `::` in the ASCII drawing, while a non-descend is indicated by the absence of those marks.

In [21]:
coffee.show_tree()

   -<--O   21dbd [       1 Item(s)] Morning coffee
    [   1 unit] Morning coffee
       | -<--#:: 39ac2 [       1 m3] Coffee
       |        Stage: Sugar
       | -<--B*  8233b [       1 kg] Sugar
       x 


In [22]:
# interactively specify descent
next(coffee.children_with_flow(fg['flow_coffee'])).term.descend=False

In [23]:
# the `::` have gone away
coffee.show_tree()

   -<--O   21dbd [       1 Item(s)] Morning coffee
    [   1 unit] Morning coffee
       | -<--#   39ac2 [       1 m3] Coffee
       |        Stage: Sugar
       | -<--B*  8233b [       1 kg] Sugar
       x 


In [24]:
fg.get('morning_coffee').fragment_lcia(qs[4]).show_components()

[local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA] kg CO2 eq
------------------------------------------------------------
     0.254 =       1.83 x      0.139 | 39ac2     0.00033 [ Input] -#   coffee_1_cup
   0.00794 =      0.015 x      0.529 | 8233b       0.015 [ Input] -B*  market for sugar, from sugarcane [GLO]
     0.262 [local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA]


# Shower
This shows how the fragment structure, plus the use of dynamically-computed balancing flows, can be used to implement a parametric calculation

In [25]:
# First- a model for heat losses in the water heater-- Assume 35% of heat is lost 
heat = B.new_link('Lossy Heat', 'net calorific value', 'Output', 0.65, units='MJ')
B.new_link('Delivered heat', 'net calorific value', 'Input', 1.0, units='MJ', parent=heat).terminate(B.terms('ng_mj'))

heat.show_tree(True)

Naming fragment 23f97db9-8141-4547-8f8f-d534cc7fdd07 -> lossy_heat
   -<--O   23f97 [*   0.65 MJ] Lossy Heat
    [   1 unit] Lossy Heat
       |        Stage: heat production, natural gas, at boiler atmospheric non-modulating <100kW
       | -<--B*  b100f [       1 MJ] Delivered heat
       x 


In [26]:
#
hw = B.new_link('Hot water 50C', 'volume', 'Output', 1.0, units='l')
# hot water, in volume
B.new_link('water, at user', 'volume', 'Input', balance=True, parent=hw, stage='Tap water').terminate(B.terms('tap_water'))
# heat required to warm 1 liter of water by 1 degree C
kj = B.new_link('kj per kg water degree C', 'net calorific value', 'Input', 4.184, units='KJ', parent=hw)
# 25 degrees C
kjc = B.new_link('times degrees C', 'net calorific value', 'Input', 25, parent=kj, name='_hot_water_delta_t')  # need ad hoc scenarios
# we terminate our heat flow to our lossy heat proces
B.new_link('Lossy heat in boiler', 'net calorific value', 'Input', balance=True, parent=kjc).terminate(heat)


hw.show_tree(True)

Naming fragment d2f73ac1-5cc1-4d25-a71a-838e854a336e -> hot_water_50c
Naming fragment 4b8d27dc-0179-4311-a87c-03c919363591 -> _hot_water_delta_t
   -<--O   d2f73 [*  0.001 m3] Hot water 50C
    [   1 unit] Hot water 50C
       | -<--O   bb72b [*0.00418 MJ] kj per kg water degree C
       |  [   1 unit] kj per kg water degree C
       |     | -<--O   4b8d2 [*     25 MJ] times degrees C
       |     |  [   1 unit] times degrees C
       |     |     | -<--#:: 558d2 (=      0 MJ) Lossy heat in boiler
       |     |     x 
       |     x 
       |        Stage: Tap water
       | -<--B*  83a67 (=      0 m3) Water, at user
       x 


In [27]:
# and the shower itB equals a mix of hot+cold water, plus soap
shower = B.new_link('Hot Shower', 'Count', 'Output', 1.0)
mins = B.new_link('Minutes per shower', 'Count', 'Input', 15, parent=shower)
ls = B.new_link('liters per minute', 'volume', 'Input', 8, units='l', parent=mins)
cold = B.new_link('cold water fraction', 'volume', 'Input', 0.1, flow_ref='flow_water_at_user', parent=ls,
                  stage='Cold water')
cold.terminate(B.terms('tap_water'))
# note "descend=False"
B.new_link('hot water fraction', 'volume', 'Input', balance=True, parent=ls, stage='Hot water').terminate(hw, descend=False)

B.new_link('some soap', 'mass', 'Input', 0.025, parent=shower, stage='Soap').terminate(B.terms('soap'))

shower.show_tree(True)

Naming fragment b29cfdf7-c7ca-45d1-b4f1-744de5060845 -> hot_shower
   -<--O   b29cf [       1 Item(s)] Hot Shower
    [   1 unit] Hot Shower
       | -<--O   b0ff2 [*     15 Item(s)] Minutes per shower
       |  [   1 unit] Minutes per shower
       |     | -<--O   5add9 [*  0.008 m3] liters per minute
       |     |  [   1 unit] liters per minute
       |     |     |        Stage: Cold water
       |     |     | -<--B*  1e88c [*    0.1 m3] Water, at user
       |     |     |        Stage: Hot water
       |     |     | -<--#   1caed (=      0 m3) hot water fraction
       |     |     x 
       |     x 
       |        Stage: Soap
       | -<--B*  24fd3 [*  0.025 kg] some soap
       x 


## Shower traversal


In [28]:
_=enum(shower.traverse(observed=True))

 [00] b29cf           1 [ Input] -O   hot_shower
 [01] b0ff2          15 [ Input] -O   Minutes per shower
 [02] 5add9        0.12 [ Input] -O   liters per minute
 [03] 1e88c       0.012 [ Input] -B*  market for tap water [RoW]
 [04] 1caed       0.108 [ Input] -#   hot_water_50c
 [05] 24fd3       0.025 [ Input] -B*  market for soap [GLO]


## Shower LCIA

In [29]:
shower.fragment_lcia(qs[4]).show_components()

completed 66 iterations
completed 63 iterations
[local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA] kg CO2 eq
------------------------------------------------------------
      1.43 =        108 x     0.0132 | 1caed       0.108 [ Input] -#   hot_water_50c
    0.0484 =      0.025 x       1.93 | 24fd3       0.025 [ Input] -B*  market for soap [GLO]
    0.0122 =         12 x    0.00101 | 1e88c       0.012 [ Input] -B*  market for tap water [RoW]
      1.49 [local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA]


## Scenarios and parameterization
Observations can be made under a specific scenario, which is then specified during traversal or LCIA computation.

In [30]:
fg.observe(mins, exchange_value=20, scenario='long shower')
fg.observe(cold, exchange_value=0, scenario='hot shower')

'coffee_demo/1e88cb2e-db95-4064-8169-caf50b1081cb'

In [31]:
shower.fragment_lcia(qs[4], 'long shower').show_components()

[local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA] kg CO2 eq
------------------------------------------------------------
       1.9 =        144 x     0.0132 | 1caed       0.144 [ Input] -#   hot_water_50c
    0.0484 =      0.025 x       1.93 | 24fd3       0.025 [ Input] -B*  market for soap [GLO]
    0.0162 =         16 x    0.00101 | 1e88c       0.016 [ Input] -B*  market for tap water [RoW]
      1.97 [local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA]


In [32]:
shower.fragment_lcia(qs[4], 'hot shower').show_components()

[local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA] kg CO2 eq
------------------------------------------------------------
      1.59 =        120 x     0.0132 | 1caed        0.12 [ Input] -#   hot_water_50c
    0.0484 =      0.025 x       1.93 | 24fd3       0.025 [ Input] -B*  market for soap [GLO]
      1.63 [local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA]


In [33]:
shower.fragment_lcia(qs[4], ('long shower', 'hot shower')).show_components()

[local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA] kg CO2 eq
------------------------------------------------------------
      2.11 =        160 x     0.0132 | 1caed        0.16 [ Input] -#   hot_water_50c
    0.0484 =      0.025 x       1.93 | 24fd3       0.025 [ Input] -B*  market for soap [GLO]
      2.16 [local.lcia.traci.2.1] Global Warming Air [kg CO2 eq] [LCIA]


# The end